In [1]:
#Importando librerias
import pandas as pd
import numpy as np
from urllib.request import FancyURLopener
from bs4 import BeautifulSoup
import requests
import urllib

from urllib.request import urlopen
from urllib.request import urlopen, Request
from bs4 import BeautifulSoup
import time
from random import choice
from urllib.error import HTTPError 
import urllib.parse

Extracción y Preparación de la Data

In [2]:
#Extrayendo la data para análisis
#NOTA: PARA JALAR LA DATA DE OTRA PC, CAMBIAR LA RUTA DE s A LA NUEVA RUTA
s=r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\train.xlsx'
archivo=pd.ExcelFile(s)
hoja=archivo.sheet_names

In [3]:
df=archivo.parse('Hoja de datos')
df.head()

,Category,Topic,Source,Headline,Text,Link
0,Fake,Education,El Ruinaversal,"RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...","RAE INCLUIRÁ LA PALABRA ""LADY"" EN EL DICCIONAR...",http://www.elruinaversal.com/2017/06/10/rae-in...
1,Fake,Education,Hay noticia,"La palabra ""haiga"", aceptada por la RAE","La palabra ""haiga"", aceptada por la RAE La Rea...",https://haynoticia.es/la-palabra-haiga-aceptad...
2,Fake,Education,El Ruinaversal,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,YORDI ROSADO ESCRIBIRÁ Y DISEÑARÁ LOS NUEVOS L...,http://www.elruinaversal.com/2018/05/06/yordi-...
3,True,Education,EL UNIVERSAL,UNAM capacitará a maestros para aprobar prueba...,UNAM capacitará a maestros para aprobar prueba...,http://www.eluniversal.com.mx/articulo/nacion/...
4,Fake,Education,Lamula,pretenden aprobar libros escolares con conteni...,Alerta: pretenden aprobar libros escolares con...,https://redaccion.lamula.pe/2018/06/19/memoria...


In [4]:
df.shape

(676, 6)

In [5]:
#obtener solo las paginas http
df_link=df.drop(['Category','Topic','Source','Headline','Text'] ,axis=1)
#Convirtiendo la data http de tipo objeto a string para uso en URLopener
df_link=df_link['Link'].astype(str)

In [6]:
#Prueba del link
textp=df_link.loc[3]
textp

'http://www.eluniversal.com.mx/articulo/nacion/sociedad/2017/02/8/unam-capacitara-maestros-para-aprobar-prueba-pisa'

In [7]:
#Colocando distintos agentes de búsqueda
user_agents = [
    'Mozilla/5.0 (Windows; U; Windows NT 5.1; it; rv:1.8.1.11) Gecko/20071127 Firefox/2.0.0.11',
    'Opera/9.25 (Windows NT 5.1; U; en)',
    'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)',
    #'Mozilla/5.0 (compatible; Konqueror/3.5; Linux) KHTML/3.5.5 (like Gecko) (Kubuntu)',
    #'Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.8.0.12) Gecko/20070731 Ubuntu/dapper-security Firefox/1.5.0.12',
    #'Lynx/2.8.5rel.1 libwww-FM/2.14 SSL-MM/1.4.1 GNUTLS/1.2.9'
]

Funcion de extraccion total no se usa FancyURLopener porque bloqueaba la descarg
Se cambia de urlpone y urllib.request
Se agrega excepción para errores de paginas http inexistentes, se coloca "NA" en esos campos

In [11]:
def agrupa_textos(df_link,rango):
    grupo=[]
    #class MyOpener(FancyURLopener,object):
        #version = choice(user_agents)
    #myopener=MyOpener()
    agent=choice(user_agents)
    
    head = {'User-Agent': agent}

    
    for i in range(0,rango):
                  
        texto= df_link_part1.loc[i]
        try:
            html = urlopen(texto)
        except:
            ptag="NA"
        else:
            #texto_html=myopener.open(urllib.parse.unquote(texto)).read()
            req  =urllib.request.Request(texto, data=None,headers=head)
            response = urllib.request.urlopen(req)
            texto_html=response.read()
            soup=BeautifulSoup(texto_html,"lxml")
            ptag=soup.findAll('p')
        grupo.append(ptag)
           
              
    return(grupo)
    
    

In [14]:
#Inicalizar el grupo, obtener la cantidad de descarga
grupo2=[]
total=len(df_link)-1

Llamada a descarga de archivo de entrenamiento, se realizan descargas por lotes de 10 con intervalos de 1 minuto

In [15]:
parcial=int(total/10)
inicio=0
for i in range(0,parcial):
    cantidad=inicio+10
    df_link_part1 = df_link.iloc[inicio:cantidad]
    df_link_part1 = df_link_part1.reset_index(drop=True)
    
    rango=cantidad-inicio
    
    grupo=agrupa_textos(df_link_part1,rango)
    for s in range(0,rango):
        grupo2.append(grupo[s])
    inicio=inicio+rango
    time.sleep(60)
    
    print(i,"  ",len(grupo2))


0    10
1    20
2    30
3    40
4    50
5    60
6    70
7    80
8    90
9    100
10    110
11    120
12    130
13    140
14    150
15    160
16    170
17    180
18    190
19    200
20    210
21    220
22    230
23    240
24    250
25    260
26    270
27    280
28    290
29    300
30    310
31    320
32    330
33    340
34    350
35    360
36    370
37    380
38    390
39    400
40    410
41    420
42    430
43    440
44    450
45    460
46    470
47    480
48    490
49    500
50    510
51    520
52    530
53    540
54    550
55    560
56    570
57    580
58    590
59    600
60    610
61    620
62    630
63    640
64    650
65    660
66    670


Conversión del Texto a Base de Datos. Formalmente se convierte en Filas con 83 columnas de separacón
Se entregan dos dataframes: El de 83 columnas y otro de 1 sola columna con Join

In [ ]:
df_textos_train=pd.DataFrame(grupo2)
df_textos_train.shape
df_texto2=pd.DataFrame(columns=['textos'])
df_texto2['textos'] = df_textos_train.apply(lambda x: ','.join(set(x.astype(str))), axis=1)

In [ ]:
#Probando la data
print(df_texto2.loc[0])


Primer Entregable: Archivos en csv con los textos de las paginas html descargadas. 
Segunda parte: Limpieza:
1.- juntar Columna de Verificación (Noticia Real, Noticia Falsa), eliminar filas con https errados
2.- Retirar textos de formato y de indicación fuera del contexto de la notica
3.- Eliminar artículos y otras palabras irrelevantes para la vectorización. 


In [ ]:
df_textos_train.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textoseparado_train.csv',encoding='utf-8')
df_texto2.to_csv(r'C:\Users\Jorge Pablo\Documents\PUCP\II_MineriaWeb\Trabajo\FakeNewCorpusSpanish_V1\textojunto_train.csv',encoding='utf-8')